# Implementation of the Algorithm Using Torch

Translate everything into torch

Implement the graphs in notion

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.spatial.distance import hamming
import torch

In [ ]:
syndata = pd.read_csv('synData9.csv')
realdata = pd.read_csv('kag_risk_factors_cervical_cancer.csv')

In [ ]:
syndataA = pd.read_csv('synData9.csv')
realdataA = pd.read_csv('kag_risk_factors_cervical_cancer.csv')

In [ ]:
med = ['Age', 'Number of sexual partners', 'First sexual intercourse',
        'Num of pregnancies', 'Smokes (years)','Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)',
        'STDs (number)', 'STDs: Number of diagnosis',
        'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis']

bin = ['Smokes', 'Hormonal Contraceptives', 'IUD', 'STDs', 'STDs:condylomatosis',
        'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
        'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
        'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
        'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
        'STDs:Hepatitis B', 'STDs:HPV','Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
        'Citology', 'Biopsy']

In [ ]:
def impute(data, graph=False):
    temp_data = data
    temp_data[temp_data == '?'] = np.nan

    for i in temp_data.columns:
        if(temp_data[i].dtype == 'O'):
            temp_data[i] = temp_data[i].astype('float')

    med = ['Age', 'Number of sexual partners', 'First sexual intercourse',
        'Num of pregnancies', 'Smokes (years)','Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)',
        'STDs (number)', 'STDs: Number of diagnosis',
        'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis']

    for i in med:
        imp = float(temp_data[i].median())
        temp_data[i].fillna(imp, inplace=True)
        temp_data[i].replace(np.nan, imp)

    bin = ['Smokes', 'Hormonal Contraceptives', 'IUD', 'STDs', 'STDs:condylomatosis',
        'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
        'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
        'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
        'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
        'STDs:Hepatitis B', 'STDs:HPV','Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
        'Citology', 'Biopsy']

    for i in bin:
        imp = float(temp_data[i].mode())
        temp_data[i].fillna(imp, inplace=True)
        temp_data[i].replace(np.nan, imp)

    if graph:
        temp_data.hist(figsize=(25,15))
    
    return temp_data

In [ ]:
def norm(data1):
    """
    Min-Max normalizes the continuous data
    """
    new_dat = data1.copy()
    for col in new_dat.columns:
        if col in med:
            denom = new_dat[col].max() - new_dat[col].min()
            if denom != 0:
                new_dat[col] = (new_dat[col] - new_dat[col].min()) / denom
            else:
                new_dat[col] = 0
    return new_dat

In [ ]:
realdata = impute(realdataA, True)
syndata = impute(syndataA, True)

#For syndata
syndataC = norm(syndata)
#For realdata
realdataC = norm(realdata)

#They both should be False, this is important
syndataC.isnull().values.any(), realdataC.isnull().values.any()

In [ ]:
def compute_distance(df1, df2):
    
    # df1 and df2 are the two dataframes, real and synthetic, after normalizing
    # the non-binary variables

    ar1 = df1.to_numpy()
    ar2 = df2.to_numpy()
    x1 = torch.Tensor(ar1)
    x2 = torch.Tensor(ar2)

    # This creates a Tensor such that each channel contains the DF.
    #The last index is the size (amount of patients) in the other DF.
    y1 = x1.unsqueeze(2).expand(x1.size()[0], x1.size()[1], x2.size()[0])

    # This creates a Tensor such that each channel contains 1 patient repeated.
    # The last index is the size (amount of patients) in the other DF.
    y2 = x2.unsqueeze(2).expand(x2.size()[0], x2.size()[1], x1.size()[0]).permute(2,1,0)

    # Specify weights for each variable
    weights = torch.ones(x1.size()[1])

    # This creates a tensor such that channel i is the distance between patient
    # i in the df2 dataframe with all patients in the df2 dataframe.
    Δy = torch.abs(y1-y2)
    result = torch.matmul(Δy.permute(0,2,1), weights).numpy()
    return result

In [ ]:
xq = compute_distance(df1=realdataC, df2=syndataC)
xq.shape

In [ ]:
newL = []
for i in range(100):
    newL.append(xq[i][:100].tolist())

In [ ]:
synD = [str(i+1) for i in range(100)]
realD = [str(i+1) for i in range(100)]

heatscores=newL

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(heatscores, cmap=mpl.cm.get_cmap('cividis_r'))

ax.set_xticks(np.arange(len(realD)), labels=realD)
ax.set_yticks(np.arange(len(synD)), labels=synD)



plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")


ax.set_title("Syn vs Real")

plt.colorbar(im, ax=ax)


plt.show()

In [ ]:
heatscores2 = newL

In [ ]:
def rank(ar):
    """
    Takes in an array, ranks it and returns it
    """
    old_ar = ar
    sorted_ar = sorted(ar)
    for i in range(len(ar)):
        ind_in_old = old_ar.index(sorted_ar[i])
        old_ar[ind_in_old] = i


    return old_ar

def synrank(ar):
    """
    The ranking algo with changing shape of arrays
    """
    new_ar = []
    for j in range(len(ar[0])):
        curr_ar = []
        for i in range(len(ar)):        
            curr_ar.append(ar[i][j])
        new_ar.append(curr_ar)

    
    return new_ar

In [ ]:
dictR = {}
dictS = {}

keys = range(100)
for i in keys:
    dictR[i] = rank(heatscores2[i])
    dictS[i] = rank(synrank(heatscores2)[i])

In [ ]:
dictS

In [ ]:
from matching.algorithms import galeshapley
matching = galeshapley(dictS, dictR)
matching